# Archon HEB WAGO

This notebook is a sandbox demonstrating how to communicate with a WAGO Modbus/TCP system using [PyModbus](http://github.com/riptideio/pymodbus).  See the [PyModbus documentation](http://pymodbus.readthedocs.io) for details.  For this implementation, we are using PyModbus as a synchronous TCP client. At the time of this
notebook, PyModbus v3.6.4 is the latest version.

The examples used in this sandbox are for a [WAGO PLC](http://www.wago.com) configured as 
a prototype Archon HEB controller node.  

Like all of our WAGO-based boxes, the sextant controller box uses a WAGO [750-891](https://www.wago.com/us/controllers-bus-couplers-i-o/controller-ethernet/p/750-891) TCP Modbus controller (3rd gen).  The
controller box has 3 Modbus I/O modules attached:
 * [750-471](https://www.wago.com/us/controllers-bus-couplers-i-o/4-channel-analog-input/p/750-471) 4-channel analog input module used to readout the germanium quad cell
 * [750-450](https://www.wago.com/us/controllers-bus-couplers-i-o/4-channel-analog-input/p/750-450) 4-channel analog input resistance measurement module connected to 1 or more platinum RTD temperature sensors (1 in the prototype box)
 * [750-530](https://www.wago.com/us/controllers-bus-couplers-i-o/8-channel-digital-output/p/750-530) 8-channel digital output module (24V/0.5A) that will be used for box-level systems power control
 

In [ ]:
from time import sleep
from datetime import datetime, date, time, timedelta

# Setup the Modbus I/O client as a synchronous TCP client

#from pymodbus.client.sync import ModbusTcpClient as mbc
from pymodbus.client import ModbusTcpClient as mbc

import numpy as np

## HEB Controller Setup

The HEB controller has one IP addresses for the WAGO Modbus/TCP fieldbus controller

We also need to set the addresses of the I/O modules.  Internally WAGO module addresss are 40xxx, but with PyModbus,we address via a logical address, logical=physical-40001, as follows:
 * 750-471 4-channel analog input module for the quad cell, Address = 40001, logical address `qcAddr=0`
 * 750-450 4-channel RTD readout: Address 40005 = logical address `rtdAddr=4`
 * 750-530 8-channel digital output: Address 40513 = logical address `doAddr=512

### HEB host aliases

Select your host from the table below, or hardwire an IP address

In [ ]:
hebHosts = {"mods1b":"192.168.139.142",
            "mods1r":"192.168.139.141",
            "mods2b":"192.168.139.242",
            "mods1b":"192.168.139.241",
            "spare1":"192.168.139.41",
            "spare2":"192.168.139.42"}

### Select host and setup devices

In [ ]:
unitID = "spare1"
hebHost = hebHosts[unitID]

# WAGO module base addresses

qcAddr = 0
rtdAddr = 4
doAddr = 512

# Number of RTDs attached

numRTDs = 2
rtdName = ['HEBTemp','DEWTemp']

# Number of digital outputs attached

numOut = 2

# Output state truth table

outName = ['Archon','Ion Gauge']
outTrue = ['ON','ON']
outFalse = ['OFF','OFF']

# Utility & Convenience Functions

The different types of sensors we use with the WAGO have different ways to convert the raw 16-bit datum received from the WAGO into physical units.  Some of these functions implement those conversions, others encapsulate the low-level WAGO interaction to make readout of sensors more convenient and include (rudimentary) exception capture.

### Convert Pt RTD into degrees Celsius

This function converts RTD output to degrees C for a platinum RTD read using a WAGO analog input module.  The temperature
resolution is 0.1$^\circ$C per ADU, and the temperature range is $-$273$^\circ$C to +850$^\circ$C.  The 16-bit digital number wraps below 0$^\circ$C to $2^{16}-1$ADU.  This handles that conversion.

In [ ]:
def ptRTD2C(rawRTD):
    tempRes = 0.1   # module resolution is 0.1C per ADU
    tempMax = 850.0 # maximum temperature for a Pt RTD in deg C
    wrapT = tempRes*((2.0**16)-1) # ADU wrap at <0C to 2^16-1

    temp = tempRes*rawRTD
    if temp > tempMax:
        temp -= wrapT

    return temp

### Read RTD temperatures

Read RTDs on a WAGO 750-450 4-channel analog input (AI) module. Returns a list with temperatures in degrees C.  

Arguments:
 * client = modbus tcp client instance
 * addr = starting address (logical) of the WAGO register with the 750-450 RTD module.
 * num = number of RTDs to read.

Returns:
 * status = True on success, False on errors
 * temp = list of temperatures
 
On read faults returns -999.99 (a nonsense temperature) in the list.

In [ ]:
def getRTDs(client,addr,num):
    temp = []
    try:
        rd = client.read_input_registers(addr,num)
    except Exception as ex:
        print(f"[{datetime.utcnow().isoformat()}] *** Warning: Cannot read WAGO RTD module - {ex}")
        for i in range(num):
            temp.append(-999.99)
        return False,temp

    for i in range(num):
        temp.append(ptRTD2C(float(rd.registers[i]))) # convert to deg C
       
    return True,temp

## Read the Quad Cell register

The quad cell board analog outputs are read using the WAGO 750-471 quad analog input board.

Address is `qcAddr`

Returns list `qcData` with the decimal quad cell raw data

In [ ]:
def getQCs(client,addr):
    numAI = 4
    qcData = []
    try:
        rd = client.read_holding_registers(addr,numAI)
    except Exception as ex:
        print(f"[{datetime.utcnow().isoformat()}] *** Warning: Cannot read quad cell board - {ex}")
        for i in range(numAI):
            qcData.append(-999.99)
        return False,qcData
    
    qcData = rd.registers
    return True,qcData

def qc2vdc(rawQC):
    posMax = 2**15 - 1
    negMin = 2**16 - 2
    if rawQC > posMax:
        Vout = 10.0*((rawQC-negMin)/posMax)
    else:
        Vout = 10.0*(rawQC/posMax)
    return Vout
        

### Read Digital Output Registers

Read the status of the digital output registers on a WAGO n-channel digital output (DO) module, returning a list
of booleans (True/False) as to their state (On/Off)

Arguments:
 * client = modbus tcp client instance
 * addr = starting register address (logical) of the WAGO DO module
 * num = number of outputs to read

Returns:
 * status = True on success, False on errors
 * states = list of booleans indicating output state
 
On read faults returns False for all states as a placeholder.

In [ ]:
def getDOStatus(client,addr,num):
    states = []
    try:
        rd = client.read_coils(addr,num)
    except Exception as ex:
        print(f"[datetime.utcnow().isoformat()] ** Warning: Cannot read WAGO DO module - {ex}")
        for i in range(num):
            states.append(False)
        return False,states
        
    states = rd.bits
    return True,states

### Set a single Digital Output channel

Set the state of a single output on a WAGO n-channel digital output (DO) module

Arguments:
 * client = modbus tcp client instance
 * baseAddr = register base address (logical) of the WAGO DO module
 * channel = channel of the module (1 to N)
 * state = state to set (True or False)
 * delay = time delay in seconds after the register write (default 0.1sec)

Returns:
 * status = True on success, False on errors

The time delay is because there is a finite time that elapses between when the WAGO PLC acknowledges it got a 
valid write and when it actually gets executed.  This delay helps break up race conditions.  The default of 0.1sec
is about the minimum you need, increase if experiments with real hardware show it is required.

In [ ]:
def setDO(client,baseAddr,channel,state,delay=0.1):
    doReg = baseAddr+channel-1
    try:
        rq = client.write_coil(doReg,state)
        sleep(delay) # inject deplay in sec 
        return True
    except Exception as ex:
        print(f"[datetime.utcnow().isoformat()] ** Warning: Cannot set WAGO channel {chan} on DO module - {ex}")
        return False

## Read the Pt RTDs

This example opens a connection to a WAGO unit, reads the Pt RTDs connected to a 4-channel analog input (AI) module, then closes the connection.

This particular example reads two platinum RTDs in the sextant ontroller box.  The 750-450
4-channel analog input module is connected to 4 Pt RTDs and has base address 40009 corresponding
to logical address 8.  Only the first 2 of 4 inputs are connected to Pt RTDs.

In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(1.0) # enough time for the TCP connection to complete

if haveConnect:
    print(f"Connected to HEB WAGO unit on IP {hebHost}")
    status,rtdTemps = getRTDs(wagoClient,rtdAddr,numRTDs)
    if status:
        print("Pt RTDs:")
        for i, temp in enumerate(rtdTemps):
            print(f"  RTD {i+1}: T={rtdTemps[i]:.1f} C")
    else:
        print("could not read the HEB WAGO")
else:
    print(f"**ERROR: Cannot connect to the WAGO on IP {hebHost}")
    
# always close the connection!

wagoClient.close()

## Read the Quad Cell

This example opens a connection to a WAGO unit, reads the outputs of the analog quad cell
preamp board connected to a 4-channel analog input (AI) module, then closes the connection.


In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(1.0) # enough time for the TCP connection to complete

if haveConnect:
    print(f"Connected to HEB WAGO unit on IP {hebHost}")
    status,qcData = getQCs(wagoClient,qcAddr)
    if status:
        print("Quad Cell:")
        for i, temp in enumerate(qcData):
            vOut = qc2vdc(qcData[i])
            print(f"  Q{i+1}: {int(qcData[i]):5d} adu = {vOut:7.4f} VDC")
    else:
        print("could not read the HEB WAGO")
else:
    print(f"**ERROR: Cannot connect to the WAGO on IP {hebHost}")
    
# always close the connection!

wagoClient.close()

## Read Digital Output register status

This example opens a connection to a WAGO unit, reads the current status of the output registers in an 8-channel digital output module, and then closes the connection.

This particular example reads the first four digital output registers used to switch power to 2 units in the HEB controller (simulated):
 * Relay 1: Archon CCD controller AC Power, Normally Open (off at box power-up)
 * Relay 2: Vacuum Ion Gauge power, Normally Open (off at box power-up)
 * Relay 3-8: not used

Normally closed means that the connected device is powered on when the HEB proper is powered on, whereas normally open means that the device is powered **off** at HEB power-up and stays
off until it is explicitly commanded to switch power on.

This truth table shows how to map register status bits (True/False) to power state:

| Output | Device     | True | False | Power-On |
|--------|------------|------|-------|----------|
|    1   | Archon     | ON   | OFF   | Open     |
|    2   | Ion Gauge  | ON   | OFF   | Open     |

On power-up of the HEB controller box and its WAGO unit, all digital outputs are False.


In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(1.0)

if haveConnect:
    print(f"Connected to WAGO unit on IP {hebHost}")
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("Digital Output Registers:")
        print(f"doState={doState}")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

## Turn on the Archon controller

This example opens a connection to a WAGO unit, sets the Archon CCD controller AC power on, 
queries the new status of the output registers, then closes the connection.

Uses the same DO module as Example 2 above, with the same truth table.

In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(0.5)

if haveConnect:
    print(f"Connected to WAGO unit on IP {hebHost}")
    
    # read the current status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("Current Register Status:")
        print(f"doState={doState}")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
        
    # sleep for 0.5 second

    sleep(0.5)
    
    # set channel 1 (Archon) True (power on)
    
    print("Setting select DO outputs to True:")
    for chan in [1,2]:
        if setDO(wagoClient,doAddr,chan,True):
            print(f"  Set {outName[chan-1]} True")
        else:
            print(f"  Could not set {outName[chan-1]} True")
    
    sleep(0.5)

    # read the new status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("New Status:")
        print(f"doState={doState}")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
    
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

## Turn the Archon off

This example opens a connection to a WAGO unit, powers off the Archon controller, 
queries the new status of the output registers, then closes the connection.

Uses the same DO module as Example 3 above, with the same truth table.

In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(0.5)

if haveConnect:
    print(f"Connected to WAGO unit on IP {hebHost}")
    
    # read the current status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("Current Register Status:")
        print(f"doState={doState}")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
        
    # sleep for 0.5 second

    sleep(0.5)
    
    # set channel 1 (Archon) False (power off
    
    print("Setting select DO outputs to False")
    for chan in [1]:
        if setDO(wagoClient,doAddr,chan,False):
            print(f"  Set {outName[chan-1]} False")
        else:
            print(f"  Could not set {outName[chan-1]} False")
    
    sleep(0.5)

    # read the new status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("New Status:")
        print(f"doState={doState}")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
    
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

## Turn off everything ("All Off")

This example opens a connection to a WAGO unit, sets all 3 digital outputs true (turns them all OFF),
queries the new status of the output registers and then closes the connection.

Uses the same DO module as Example 3 above, with the same truth table.

In [ ]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(0.5)

if haveConnect:
    print(f"Connected to WAGO unit on IP {hebHost}")
    
    # read the current status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("Current Register Status:")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
        
    # sleep for 0.5 second

    sleep(0.5)
    
    # set all channels True
    
    print("Turning outputs Off:")
    for chan in [1,2]:
        offState = False
        if outTrue[chan-1] == 'OFF':
            print('setting offState TRUE')
            offState = True
        else:
            print('setting offState FALSE')
            
        if setDO(wagoClient,doAddr,chan,offState):
            print(f"  Turned {outName[chan-1]} OFF")
        else:
            print(f"  Could not turn {outName[chan-1]} OFF")
    
    sleep(0.5)

    # read the new status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("New Status:")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
    
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

## readout QC every 1 sec

runs up to `maxReads` reads of the QC board every `readDelay` seconds.  

In [ ]:
import sys

# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# maximum number of reads

maxReads = 50

# time delay between reads

readDelay = 1.0 # seconds

# volts per ADU

vPerADU = 10.0/(2**15 - 1)

# arrays to hold data

qcList = ['QC1','QC2','QC3','QC4']
qcVout = {}
for qc in qcList:
    qcVout[qc] = []
    
# connect 

haveConnect = wagoClient.connect()
sleep(1.0) # enough time for the TCP connection to complete

if haveConnect:
    print(f"Connected to HEB WAGO unit on IP {hebHost}")
    
    for j in range(maxReads):
        status,qcData = getQCs(wagoClient,qcAddr)
        if status:
            outStr = f'[{j:4d}]'
            for i in range(len(qcData)):
                vOut = qc2vdc(qcData[i])
                outStr += f' {vOut:8.5f}'
                qcVout[qcList[i]].append(vOut)
            sys.stdout.write(f"{outStr}\r")
            sleep(readDelay)
    else:
        print("\nDone, quick stats:")
        # Finishing statistics

        for qc in qcList:
            qcMean = np.mean(qcVout[qc])
            qcStd = np.std(qcVout[qc])
            qcMed = np.median(qcVout[qc])
            print(f'{qc}: mean={qcMean:.5f} median={qcMed:.5f} std={qcStd:.5f}')
        print(f'1 adu = {vPerADU:.5f} VDC')
            
else:
    print(f'\nERROR: Could not connect to HEB WAGO on IP {hebHost}')

# always close the connection!

wagoClient.close()

In [ ]:

for qc in qcList:
    qcMean = np.mean(qcVout[qc])
    qcStd = np.std(qcVout[qc])
    qcMed = np.median(qcVout[qc])
    print(f'{qc}: mean={qcMean:.5f} median={qcMed:.5f} std={qcStd:.5f}')

print(f'1 adu = {vPerADU:.5f} VDC')
